In [1]:
# importing libraries

import PyPDF2
import urllib.request
import pandas as pd
import numpy as np
import math

In [2]:
#downloading the pdf file

pdf_path = "https://cdn.cebraspe.org.br/concursos/PETROBRAS_21_NS/arquivos/ED_8_PETROBRAS_PSP1_2021_RES_FINAL_OBJ_CONV_TITULOS.PDF"

file_name = pdf_path.split('/')[-1]
response = urllib.request.urlopen(pdf_path)    
file = open(file_name, 'wb')
file.write(response.read())
file.close()

In [3]:
#reading the pdf content

pdf_object = open(file_name, 'rb') 
pdf_reader = PyPDF2.PdfFileReader(pdf_object)

#writing the pdf content in a single string 

file_content = ''
for i in range(pdf_reader.numPages):
    number_digits = (int(math.log10(i + 1)) + 1)
    
    # erasing the page number, cleaning the text and filling the string up
    
    if number_digits == 1:
        page_content = pdf_reader.getPage(i).extractText().replace('\n', '').replace('  ', ' ')[2:]
    elif number_digits == 2:
        page_content = pdf_reader.getPage(i).extractText().replace('\n', '').replace('  ', ' ')[3:]
    else:
        page_content = pdf_reader.getPage(i).extractText().replace('\n', '').replace('  ', ' ')[4:]
    file_content += page_content
    
pdf_object.close()

# parsing the string text

parsed_file_content = file_content.split(': ')

In [4]:
#getting the datasets column names

column_names = (parsed_file_content[3].split('.')[0].split(', ')[1:-1]
                + parsed_file_content[3].split('.')[0].split(', ')[-1].split(' e '))

In [5]:
# finding all the candidates

dicionario = {}
lista_nome_enfase = []

# broad competition candidates (except Emphasis 19)

def achar_candidatos_ac(n_min, n_max):

    for i in range(n_min, n_max, 5):
        numero_enfase = parsed_file_content[i - 1].split(' ')[-2:][0] + ' ' + parsed_file_content[i - 1].split(' ')[-2:][1]
        nome_enfase = parsed_file_content[i].split('. ')[0].split(' AC ')[0]
        candidatos = parsed_file_content[i].split('. ')[0].split(' AC ')[1].split(' / ')
        dicionario[numero_enfase] = candidatos
        lista_nome_enfase.append([numero_enfase, nome_enfase])

achar_candidatos_ac(4, 40)
achar_candidatos_ac(42, 73)
achar_candidatos_ac(75, 111)

# broad competition candidates (Emphasis 19)

numero_enfase = parsed_file_content[114].split(' ')[-2:][0] + ' ' + parsed_file_content[114].split(' ')[-2:][1]
nome_enfase = parsed_file_content[115].split(')')[0]
candidatos = parsed_file_content[116].split('. ')[1].split(' / ')
dicionario[numero_enfase] = candidatos
lista_nome_enfase.append([numero_enfase, nome_enfase])

# people with disability candidates (except Emphasis 19)

def achar_candidatos_pcd(n_min, n_max):

    for i in range(n_min, n_max, 5):
        numero_enfase = parsed_file_content[i - 1].split(' ')[-2:][0] + ' ' + parsed_file_content[i - 1].split(' ')[-2:][1] + ' PCD'
        nome_enfase = parsed_file_content[i].split(' Resultado')[0]
        candidatos = parsed_file_content[i + 1].split('. ')[1].split(' / ')
        dicionario[numero_enfase] = candidatos
        lista_nome_enfase.append([numero_enfase, nome_enfase])
    
achar_candidatos_pcd(5, 36)
achar_candidatos_pcd(43, 69)
achar_candidatos_pcd(76, 112)

# people with disability candidates (Emphasis 19)

numero_enfase = parsed_file_content[116].split(' ')[-2:][0] + ' ' + parsed_file_content[116].split(' ')[-2:][1] + ' PCD'
nome_enfase = parsed_file_content[117].split(' Resultado')[0]
candidatos = parsed_file_content[118].split('. ')[1].split(' / ')
dicionario[numero_enfase] = candidatos
lista_nome_enfase.append([numero_enfase, nome_enfase])

# black people candidates (except Emphasis 19)

def achar_candidatos_negros(n_min, n_max):

    for i in range(n_min, n_max, 5):
        numero_enfase = parsed_file_content[i - 1].split(' ')[-2:][0] + ' ' + parsed_file_content[i - 1].split(' ')[-2:][1] + ' NEGROS'
        nome_enfase = parsed_file_content[i].split(' Resultado')[0]
        candidatos = parsed_file_content[i + 1].split('. ')[1].split(' / ')
        dicionario[numero_enfase] = candidatos
        lista_nome_enfase.append([numero_enfase, nome_enfase])
        
achar_candidatos_negros(7, 38)
achar_candidatos_negros(40, 71)
achar_candidatos_negros(73, 114)

# black people candidates (Emphasis 19)

numero_enfase = parsed_file_content[118].split(' ')[-2:][0] + ' ' + parsed_file_content[118].split(' ')[-2:][1] + ' NEGROS'
nome_enfase = parsed_file_content[119].split(' Resultado')[0]
candidatos = parsed_file_content[120].split('. ')[1].split(' / ')
dicionario[numero_enfase] = candidatos
lista_nome_enfase.append([numero_enfase, nome_enfase])

In [6]:
# saving as dataframe

for enfase in dicionario.keys():
    df = pd.DataFrame(dicionario[enfase])[0].str.split(',', expand = True)
    df.columns = column_names
    
    # converting numeric values to integer
    
    for column in column_names[2:]:
        df[column] = df[column].astype('float').astype('int')
    
    #sorting the candidates according to the exam's tie break rules
    
    df.sort_values(by = [column_names[9],
                         column_names[5],
                         column_names[6],
                         column_names[7],
                         column_names[3]
                        ], ascending = False, inplace = True)

    df.index = np.arange(1, len(df) + 1)
    dicionario[enfase] = df

In [7]:
# saving as an excel file

writer = pd.ExcelWriter('petrobras_2021_candidatos_aprovados.xlsx')

for enfase in dicionario:
    dicionario[enfase].to_excel(writer, enfase)

writer.save()